In [1]:
# write the list of necessary packages here:
!pip install pandas
!pip install nltk
!pip install spacy
!pip install scikit-learn
!pip install ollama
!pip install evaluate
!pip install transformers[torch] accelerate>=0.26.0


In [1]:
# importing packages here:
import numpy as np
import evaluate
import pandas as pd
import nltk
import random
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification
from datasets import Dataset
from ollama import chat
from ollama import ChatResponse
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import torch

In [2]:
# loading dataset
dataset_ori= pd.read_csv('orientation-fi-train.tsv',sep='\t')
dataset_pow=pd.read_csv('power-fi-train.tsv',sep='\t')
# checking datasets
display(dataset_ori.head(3))
display(dataset_pow.head(3))
dataset_ori.info()
dataset_pow.info()
dataset_pow['label'].value_counts()

,id,speaker,sex,text,text_en,label
0,fi00000,396f7d76936fcdda2f63bffd21da7104,F,Arvoisa rouva puhemies! Haluan myöskin osaltan...,Madam President! I would also like to join in ...,1
1,fi00001,59ca750f13ff9333feaeaf7a5f9538ec,F,Arvoisa rouva puhemies! Myös minä kiitän tästä...,Madam President! I too thank you for this deba...,1
2,fi00002,59ca750f13ff9333feaeaf7a5f9538ec,F,Arvoisa puhemies! Edustaja Haavisto esitti huo...,Mr President! Mr Haavisto expressed his concer...,1


,id,speaker,sex,text,text_en,label
0,fi03180,2241b20fc844da218ddb7f75768cdc1a,F,Arvoisa puhemies! Kiitoksia Suomen valtuuskunn...,Mr President! Thank you to the chairman of the...,1
1,fi03181,3c7c426430f1c95d746bbd001e0a561f,M,Arvoisa puhemies! Euroopan neuvosto on merkitt...,Mr President! The Council of Europe is an impo...,1
2,fi03182,13cfa1824e32176b120f49427b1f5e17,M,Arvoisa puhemies! Todellakin jännittyneen kans...,"Mr President! Indeed, because of the tense int...",0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1179 entries, 0 to 1178
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       1179 non-null   object
 1   speaker  1179 non-null   object
 2   sex      1179 non-null   object
 3   text     1179 non-null   object
 4   text_en  1049 non-null   object
 5   label    1179 non-null   int64 
dtypes: int64(1), object(5)
memory usage: 55.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6111 entries, 0 to 6110
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       6111 non-null   object
 1   speaker  6111 non-null   object
 2   sex      6111 non-null   object
 3   text     6111 non-null   object
 4   text_en  5427 non-null   object
 5   label    6111 non-null   int64 
dtypes: int64(1), object(5)
memory usage: 286.6+ KB


label
0    3386
1    2725
Name: count, dtype: int64

In [3]:
dataset_ori['label'].value_counts()

label
1    676
0    503
Name: count, dtype: int64

In [3]:
# test splits
dataset_ori=dataset_ori.dropna(subset=['text_en'])
dataset_pow=dataset_pow.dropna(subset=['text_en'])
ori_train,ori_test=train_test_split(dataset_ori,test_size=0.05)
pow_train,pow_test=train_test_split(dataset_pow,test_size=0.05)
ori_test = ori_test.reset_index()
pow_test = pow_test.reset_index()
ori_train = ori_train.reset_index()
pow_train = pow_train.reset_index()

In [4]:
print(pow_test['text'].str.len().max())
print(pow_test['text_en'].str.len().max())

print(ori_test['text'].str.len().max())
print(ori_test['text_en'].str.len().max())

pow_test.info()
ori_test.info()

9170
7092
5596
5077
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 272 entries, 0 to 271
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   index    272 non-null    int64 
 1   id       272 non-null    object
 2   speaker  272 non-null    object
 3   sex      272 non-null    object
 4   text     272 non-null    object
 5   text_en  272 non-null    object
 6   label    272 non-null    int64 
dtypes: int64(2), object(5)
memory usage: 15.0+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   index    53 non-null     int64 
 1   id       53 non-null     object
 2   speaker  53 non-null     object
 3   sex      53 non-null     object
 4   text     53 non-null     object
 5   text_en  53 non-null     object
 6   label    53 non-null     int64 
dtypes: int64(2), object(5)
memory usage: 3.0+ KB


In [ ]:
# Task 1 zero-shot inference
# using a multilingual causal language model as zero-shot manner for political oriantation
# llama needs to be up and running

originaltext_ori_correct_labels=0
engtext_ori_correct_labels=0
refused_amount_ing=0
refused_amount_org=0

for index, row in ori_test.iterrows():
  print(f"processing row {index}")
  response: ChatResponse = chat(model='llama3.2:1b', messages=[
      {
        'role': 'tool',
        'content': ('from finnish text only write "0" if political oriantation is left or write "1" if political oriantation is right finnish text starts here: '+row['text'])[:4500] + ' . \n ',
      },
    ],
    options={"temperature":0.1})
  try:
    if(int(response['message']['content'][0])==row['label']):
      originaltext_ori_correct_labels+=1
  except:
    refused_amount_org+=1
      
  if (pd.isna(row['text_en']) or row['text_en'] == ''):
    continue
  response: ChatResponse = chat(model='llama3.2:1b', messages=[
      {
        'role': 'tool',
        'content': ('from english text from finnish parliament only write "0" if political oriantation is left or write "1" if political oriantation is right english text starts here: '+str(row['text_en'])[:4500] + ' . \n '),
      },
    ],
    options={"temperature":0.1})
  try:
    if(int(response['message']['content'][0])==row['label']):
      engtext_ori_correct_labels+=1
  except:
    refused_amount_ing+=1
      
      
    
org_accuracy = originaltext_ori_correct_labels/len(ori_test)
eng_accuracy = engtext_ori_correct_labels/len(ori_test)
print(f"accuracy from original text = {org_accuracy}")
print(f"accuracy from english text = {eng_accuracy}")

print(f"amount of refused to label original lang = {refused_amount_org}")
print(f"amount of refused to label eng lang = {refused_amount_ing}")

print(f"only unrefused accuracy from original text = {originaltext_ori_correct_labels/(len(ori_test)-refused_amount_org)}")
print(f"only unrefused accuracy from english text = {engtext_ori_correct_labels/(len(ori_test)-refused_amount_ing)}")


processing row 0
processing row 1
processing row 2
processing row 3
processing row 4
processing row 5
processing row 6
processing row 7
processing row 8
processing row 9
processing row 10
processing row 11
processing row 12
processing row 13
processing row 14
processing row 15
processing row 16
processing row 17
processing row 18
processing row 19
processing row 20
processing row 21
processing row 22
processing row 23
processing row 24
processing row 25
processing row 26
processing row 27
processing row 28
processing row 29
processing row 30
processing row 31
processing row 32
processing row 33
processing row 34
processing row 35
processing row 36
processing row 37
processing row 38
processing row 39
processing row 40
processing row 41
processing row 42
processing row 43
processing row 44
processing row 45
processing row 46
processing row 47
processing row 48
processing row 49
processing row 50
processing row 51
processing row 52
accuracy from original text = 0.41509433962264153
accura

In [5]:
# Task 2 zero-shot inference
# using a multilingual causal language model as zero-shot manner for political power
# llama needs to be up and running

originaltext_pow_correct_labels=0
engtext_pow_correct_labels=0
refused_amount_org=0
refused_amount_ing=0

for index, row in pow_test.iterrows():
  print(f"processing row {index}")
  response: ChatResponse = chat(model='llama3.2:1b', messages=[
      {
        'role': 'tool',
        'content': ('from finnish text only write "0" if political party is currently governing (in power) or write "1" if political party is in opposition finnish text starts here: '+row['text'][:4500]+ ' . \n '),
      },
    ],
    options={"temperature":0.1,"timeout":5})
  try:
    if(int(response['message']['content'][0])==row['label']):
      originaltext_pow_correct_labels+=1
  except:
    refused_amount_org+=1
  
  if (pd.isna(row['text_en']) or row['text_en'] == ''):
    continue
  
  response: ChatResponse = chat(model='llama3.2:1b', messages=[
      {
        'role': 'tool',
        'content': ('from english text from finnish parliament only write "0" if political party is currently governing (in power) or write "1" if political party is in opposition english text starts here: '+str(row['text_en'])[:4500]+ ' . \n '),
      },
    ],
    options={"temperature":0.1 , "timeout":5})
  try:
    if(int(response['message']['content'][0])==row['label']):
      engtext_pow_correct_labels+=1
  except:
    refused_amount_ing+=1
      
      
    
org_accuracy=originaltext_pow_correct_labels/len(pow_test)
eng_accuracy=engtext_pow_correct_labels/len(pow_test)
print(f"accuracy from original text = {org_accuracy}")
print(f"accuracy from english text = {eng_accuracy}")

print(f"amount of refused to label original lang = {refused_amount_org}")
print(f"amount of refused to label eng lang = {refused_amount_ing}")

print(f"only unrefused accuracy from original text = {originaltext_pow_correct_labels/(len(pow_test)-refused_amount_org)}")
print(f"only unrefused accuracy from english text = {engtext_pow_correct_labels/(len(pow_test)-refused_amount_ing)}")

processing row 0
processing row 1
processing row 2
processing row 3
processing row 4
processing row 5
processing row 6
processing row 7
processing row 8
processing row 9
processing row 10
processing row 11
processing row 12
processing row 13
processing row 14
processing row 15
processing row 16
processing row 17
processing row 18
processing row 19
processing row 20
processing row 21
processing row 22
processing row 23
processing row 24
processing row 25
processing row 26
processing row 27
processing row 28
processing row 29
processing row 30
processing row 31
processing row 32
processing row 33
processing row 34
processing row 35
processing row 36
processing row 37
processing row 38
processing row 39
processing row 40
processing row 41
processing row 42
processing row 43
processing row 44
processing row 45
processing row 46
processing row 47
processing row 48
processing row 49
processing row 50
processing row 51
processing row 52
processing row 53
processing row 54
processing row 55
pr

In [4]:
# loading the pretrained model
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-multilingual-cased")
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-multilingual-cased", num_labels=2, torch_dtype="auto")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# making trainer for political oriantation it uses original language
def tokenize_function(examples):
    tokenized = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)
    tokenized["labels"] = examples["label"]  # Ensure labels are included
    return tokenized

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

train_dataset = ori_train[["text", "label"]]  # Keep only relevant columns
test_dataset = ori_test[["text", "label"]]

train_dataset = Dataset.from_pandas(train_dataset)
test_dataset = Dataset.from_pandas(test_dataset)

tokenized_ori_train = train_dataset.map(tokenize_function, batched=True)
tokenized_ori_test = test_dataset.map(tokenize_function, batched=True)

training_args = TrainingArguments(
    output_dir="test_trainer",             # Directory to save model and checkpoints
    evaluation_strategy="epoch",          # Evaluate after every epoch
    per_device_train_batch_size=16,       # Train on 16 samples per GPU per batch
    per_device_eval_batch_size=32,        # Use a larger batch size for evaluation
    num_train_epochs=6,                   # Train for 6 epochs (adjust as needed)
    learning_rate=3e-5,                   # Lower learning rate for fine-tuning
    weight_decay=0.01,                    # Regularization to prevent overfitting
    warmup_steps=500,                     # Warmup for LR scheduling
    logging_dir="logs",                   # Directory for logs
    logging_steps=100,                    # Log training metrics every 100 steps
    save_steps=500,                       # Save model every 500 steps
    save_total_limit=2,                   # Keep only the last 2 checkpoints
    fp16=True,                            # Use mixed-precision training
    report_to="none",                     # Disable reporting to external tools
    disable_tqdm=False,                   # Show progress bar
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ori_train,
    eval_dataset=tokenized_ori_test,
    compute_metrics=compute_metrics,
)
trainer.train()

Map:   0%|          | 0/1120 [00:00<?, ? examples/s]

Map:   0%|          | 0/59 [00:00<?, ? examples/s]

c:\Users\dreky\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/420 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 0.8348823189735413, 'eval_accuracy': 0.5254237288135594, 'eval_runtime': 4.2942, 'eval_samples_per_second': 13.739, 'eval_steps_per_second': 0.466, 'epoch': 5.0}
{'loss': 0.5035, 'grad_norm': 46.9262809753418, 'learning_rate': 2.3880000000000002e-05, 'epoch': 5.71}


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 0.926078736782074, 'eval_accuracy': 0.6101694915254238, 'eval_runtime': 4.1895, 'eval_samples_per_second': 14.083, 'eval_steps_per_second': 0.477, 'epoch': 6.0}
{'train_runtime': 1637.7192, 'train_samples_per_second': 4.103, 'train_steps_per_second': 0.256, 'train_loss': 0.5486969448271252, 'epoch': 6.0}


TrainOutput(global_step=420, training_loss=0.5486969448271252, metrics={'train_runtime': 1637.7192, 'train_samples_per_second': 4.103, 'train_steps_per_second': 0.256, 'total_flos': 442026573004800.0, 'train_loss': 0.5486969448271252, 'epoch': 6.0})

In [6]:
eval_results = trainer.evaluate()
print(eval_results)

  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 0.6930655241012573, 'eval_accuracy': 0.5423728813559322, 'eval_runtime': 4.2504, 'eval_samples_per_second': 13.881, 'eval_steps_per_second': 0.471, 'epoch': 3.0}


In [13]:
# making trainer for power it uses english
tokenizer2 = AutoTokenizer.from_pretrained("google-bert/bert-base-multilingual-cased")
model2 = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-multilingual-cased", num_labels=2, torch_dtype="auto")

def tokenize_function2(examples):
    tokenized = tokenizer(examples["text_en"], padding="max_length", truncation=True, max_length=128)
    tokenized["labels"] = examples["label"]  # Ensure labels are included
    return tokenized

metric2 = evaluate.load("accuracy")

def compute_metrics2(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric2.compute(predictions=predictions, references=labels)


train_dataset2 = pow_train[["text_en", "label"]]  # Keep only relevant columns
test_dataset2 = pow_test[["text_en", "label"]]

train_dataset2 = train_dataset2.dropna(subset=["text_en"]) # delete untranslated ones  
test_dataset2 = test_dataset2.dropna(subset=["text_en"])

train_dataset2 = Dataset.from_pandas(train_dataset2)
test_dataset2 = Dataset.from_pandas(test_dataset2)

tokenized_pow_train = train_dataset2.map(tokenize_function2, batched=True)
tokenized_pow_test = test_dataset2.map(tokenize_function2, batched=True)


training_args2 = TrainingArguments(
    output_dir="test_trainer2",             # Directory to save model and checkpoints
    evaluation_strategy="epoch",          # Evaluate after every epoch
    per_device_train_batch_size=16,       # Train on 16 samples per GPU per batch
    per_device_eval_batch_size=32,        # Use a larger batch size for evaluation
    num_train_epochs=6,                   # Train for 3 epochs (adjust as needed)
    learning_rate=3e-5,                   # Lower learning rate for fine-tuning
    weight_decay=0.01,                    # Regularization to prevent overfitting
    warmup_steps=500,                     # Warmup for LR scheduling
    logging_dir="logs",                   # Directory for logs
    logging_steps=100,                    # Log training metrics every 100 steps
    save_steps=500,                       # Save model every 500 steps
    save_total_limit=2,                   # Keep only the last 2 checkpoints
    fp16=True,                            # Use mixed-precision training
    report_to="none",                     # Disable reporting to external tools
    disable_tqdm=False,                   # Show progress bar
)

trainer2 = Trainer(
    model=model2,
    args=training_args2,
    train_dataset=tokenized_pow_train,
    eval_dataset=tokenized_pow_test,
    compute_metrics=compute_metrics2,
)
trainer2.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/5152 [00:00<?, ? examples/s]

Map:   0%|          | 0/275 [00:00<?, ? examples/s]

c:\Users\dreky\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/1932 [00:00<?, ?it/s]

...

  0%|          | 0/9 [00:00<?, ?it/s]

{'eval_loss': 1.1395609378814697, 'eval_accuracy': 0.6218181818181818, 'eval_runtime': 20.0829, 'eval_samples_per_second': 13.693, 'eval_steps_per_second': 0.448, 'epoch': 5.0}
{'loss': 0.1826, 'grad_norm': 25.390962600708008, 'learning_rate': 4.965083798882682e-06, 'epoch': 5.28}
{'loss': 0.1665, 'grad_norm': 72.68968963623047, 'learning_rate': 2.8701117318435755e-06, 'epoch': 5.59}
{'loss': 0.1491, 'grad_norm': 32.206302642822266, 'learning_rate': 7.751396648044693e-07, 'epoch': 5.9}


  0%|          | 0/9 [00:00<?, ?it/s]

{'eval_loss': 1.5836676359176636, 'eval_accuracy': 0.6327272727272727, 'eval_runtime': 19.5034, 'eval_samples_per_second': 14.1, 'eval_steps_per_second': 0.461, 'epoch': 6.0}
{'train_runtime': 9199.827, 'train_samples_per_second': 3.36, 'train_steps_per_second': 0.21, 'train_loss': 0.49041426724775483, 'epoch': 6.0}


TrainOutput(global_step=1932, training_loss=0.49041426724775483, metrics={'train_runtime': 9199.827, 'train_samples_per_second': 3.36, 'train_steps_per_second': 0.21, 'total_flos': 2033322235822080.0, 'train_loss': 0.49041426724775483, 'epoch': 6.0})

In [14]:
eval_results2 = trainer2.evaluate()
print(eval_results2)

  0%|          | 0/9 [00:00<?, ?it/s]

{'eval_loss': 1.5836676359176636, 'eval_accuracy': 0.6327272727272727, 'eval_runtime': 19.6338, 'eval_samples_per_second': 14.006, 'eval_steps_per_second': 0.458, 'epoch': 6.0}
